In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
# 데이터 불러오기
#train = pd.read_csv('tr_train.csv')
#test = pd.read_csv('tr_test.csv')
target = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/cust_train.csv')
submission = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/sample_submission.csv')

# Basic data preprocessing

In [54]:
# 구매금액, 구매수량을 숫자로 변경
#train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
#train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
#test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
#test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

# EDA

In [ ]:
# target과 feature간의 상관관계 분석

# Make features

In [ ]:
# CLNT_ID별로 묶어낼 수 있도록 feature를 만든다.
# categorical feature, numeric feature 등 다양하게 생각해본다.

In [7]:
features = []
features_te = []

In [8]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [9]:
# 총구매액, 구매건수, 평균구매액, 최대구매액
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,0,86500,2,43250,81000
1,1,1276000,16,79750,99000
2,2,560000,11,50909,62100
3,3,851200,8,106400,110400
4,4,125200,6,20867,37000
...,...,...,...,...,...
263099,263099,973000,112,8688,22000
263100,263100,22400,2,11200,12900
263101,263101,135600,4,33900,33900
263102,263102,59400,15,3960,9720


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,263104,31600,3,10533,13800
1,263105,314000,4,78500,120000
2,263106,49800,2,24900,29900
3,263107,88000,2,44000,49000
4,263108,124000,1,124000,124000
...,...,...,...,...,...
112755,375859,80000,3,26667,35000
112756,375860,902800,12,75233,149000
112757,375861,91800,2,45900,45900
112758,375862,92000,4,23000,23000


# Merge features

In [10]:
# CLNT_ID별로 묶일 수 있게 feature를 만든다.

data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
    
data = data.fillna(0)

In [11]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left')
    
data_te = data_te.fillna(0)

In [8]:
#data = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/data_w2v_1.csv', encoding = 'utf-8')
#data_te = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/data_te_w2v_1.csv', encoding = 'utf-8')

In [11]:
# train과 target 병합
data = pd.merge(data, target) ; data

,CLNT_ID,상품 키워드명 개수,가구,건강식품,계절가전,과일,구기/필드스포츠,남성의류,냉동식품,냉장/세탁가전,...,21,22,23,24,25,26,27,28,29,LABEL
0,0,1,0,0,0,0,0,0,0,0,...,-0.368843,-0.004185,0.208383,-0.049867,-0.231964,-0.377612,0.110913,-0.221835,0.472825,F20
1,1,9,0,2,0,0,0,0,0,0,...,-0.038498,-0.117219,0.125806,0.296538,-0.013347,-0.230724,0.294378,-0.194947,-0.104066,F30
2,2,6,0,0,0,0,0,0,0,0,...,-0.165663,-0.065437,0.177161,0.000899,-0.158791,-0.002563,0.178092,-0.430584,-0.410086,F30
3,3,4,0,0,0,0,0,0,0,0,...,-0.217013,0.208528,-0.166547,-0.098981,-0.329463,-0.003071,0.547893,-0.376096,0.097258,F20
4,4,4,0,0,0,0,0,4,0,0,...,-0.203696,-0.212453,-0.131294,-0.031168,-0.045855,-0.182355,0.263116,-0.283592,-0.104645,F20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263099,263099,7,0,0,0,0,0,112,0,0,...,-0.241297,-0.509127,0.045394,-0.482842,0.012604,-0.349621,0.179193,0.041320,-0.331822,M30
263100,263100,2,0,0,0,0,0,1,0,0,...,-0.245052,-0.217584,-0.063629,-0.252934,0.088624,-0.203703,0.023362,-0.212951,-0.112562,F40
263101,263101,2,0,0,0,0,0,0,0,0,...,-0.615322,0.232261,-0.070095,-0.247822,-0.213604,-0.280830,0.047617,-0.396430,0.075111,F40
263102,263102,5,0,0,0,0,0,0,0,0,...,-1.060696,-0.273748,-0.069810,0.245581,0.114618,-0.280226,0.200379,0.207308,0.577111,F30


# Scaling

In [15]:
# 로그변환, 정규화, 표준화 ...

# Feature selection

In [12]:
# permutation importance를 활용한 피처 선택
from sklearn.linear_model import LogisticRegression
from eli5.sklearn import PermutationImportance
import eli5
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# function
def FeatureSelection(df, target):
    meta = LGBMClassifier(random_state = 0)
    perm = PermutationImportance(meta.fit(df, target)).fit(df, target)
    
    perm_features_df = eli5.explain_weights_df(perm, feature_names = df.columns.tolist())
    perm_features = perm_features_df.loc[perm_features_df['weight'] >= 0.01]['feature'].tolist()
    
    model = LGBMClassifier(random_state = 0)
    cv_scores = []

    sscv = StratifiedKFold(n_splits = 5, random_state = 0)

    for p in tqdm(range(5,100,1)): 
        nf = int(perm_features_df.shape[0] * p * 0.01)
        features_selected = perm_features_df.iloc[:nf].feature
        X_new = df[features_selected]
        cv_score = cross_val_score(model, X_new, target, scoring='neg_log_loss', cv=sscv).mean()
        cv_scores.append((p,cv_score))
        
    best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
    print(best_score)
    plt.plot([k for k, _ in cv_scores], [score for _, score in cv_scores])
    plt.xlabel('Percent of features')
    plt.grid()
    plt.show()
    
    nf = int(perm_features_df.shape[0] * best_score[0] * 0.01)
    return perm_features_df.iloc[:nf].feature

In [14]:
FeatureSelection(X_train, y_train)

 35%|████████████████████████████▍                                                     | 33/95 [15:57<29:59, 29.02s/it]


KeyboardInterrupt: 

# Data split

In [15]:
from sklearn.model_selection import train_test_split
X = data.loc[:, '상품 키워드명 개수' : '29']
y = data.LABEL
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

In [36]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# 데이터 스케일링(정규분포)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_train = pd.DataFrame(X_train)
X_valid = pd.DataFrame(X_valid)

In [53]:
# 데이터 스케일링
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_train = pd.DataFrame(X_train)
X_valid = pd.DataFrame(X_valid)

# Modeling

In [13]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state = 0)
rf = RandomForestClassifier(random_state = 0)
xgb = XGBClassifier(random_state = 0)

In [37]:
# fit
lgbm.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Model optimization

In [53]:
# GridSearchCV, RandomizedSearchCV, bayes-opt 등으로 모델의 하이퍼파라미터를 최적화한다.

In [38]:
# predict
lgbm_pred = lgbm.predict_proba(X_valid)
rf_pred = rf.predict_proba(X_valid)
xgb_pred = xgb.predict_proba(X_valid)

In [39]:
# evaluate
from sklearn.metrics import log_loss

print(f"LGBMClassifier's score : {log_loss(pd.get_dummies(y_valid), lgbm_pred)}")
print(f"RandomForestClassifier's score : {log_loss(pd.get_dummies(y_valid), rf_pred)}")
print(f"XGBClassifier's score : {log_loss(pd.get_dummies(y_valid), xgb_pred)}")

LGBMClassifier's score : 1.2024565619701317
RandomForestClassifier's score : 1.35512556021945
XGBClassifier's score : 1.2068547534813046


##### Bayesian-optimization
- http://sanghyukchun.github.io/99/

In [21]:
!pip install bayesian-optimization

In [21]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import log_loss

In [22]:
### LGBM bug
### feature name이 non_ASCII인경우 버그가 발생함.
### 따라서 feature name을 전부 ASCII로 바꿔주어야 함.
X_train.columns = [i for i in range(X_train.shape[1])]
X_valid.columns = [i for i in range(X_train.shape[1])]
data_te = data_te.loc[:,'상품 키워드명 개수':]
data_te.columns = [i for i in range(X_train.shape[1])] 

In [23]:
# Step1. 탐색하고싶은 하이퍼파라미터의 범위를 dictionary 형태로 지정.
pbounds = { 'learning_rate': (0.05, 1.5),
            'n_estimators': (50, 500),
            'max_depth': (5,15),   
            'subsample': (0.8, 0.95),  
            'gamma': (0, 5)}

# Step2. 1에서 생성한 dictionary의 key를 arguments로 하는 함수 생성
def lgbm_opt(learning_rate, n_estimators, max_depth, subsample, gamma):

    # Step3. 탐색하고싶은 하이퍼파라미터의 조건 지정
    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'subsample': subsample,
        'gamma': gamma,
        'n_jobs' : -1
    }
    
    # Step4. 모델 생성
    lgbm = LGBMClassifier(**params)
    lgbm.fit(X_train, y_train)
    
    # Step5. 최대화하려는 score 선언
    score = log_loss(y_valid, lgbm.predict_proba(X_valid))
    
    return -1 * score

In [40]:
# Step6. BayesianOptimization 객체 생성
BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=1) # 최대화하려는 함수 f, 탐색범위 pbounds

# Step7. 최대화
BO_lgbm.maximize(init_points=50, n_iter=50)

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  1        | -23.73    |  2.085    |  1.094    |  5.001    |  186.0    |  0.822    |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  2        | -1.417    |  0.4617   |  0.3201   |  8.456    |  228.5    |  0.8808   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  3        | -22.61    |  2.096    |  1.044    |  7.045    |  445.2    |  0.8041   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^m

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  29       | -1.234    |  2.781    |  0.2479   |  5.599    |  104.6    |  0.8067   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  30       | -1.568    |  0.5375   |  0.3773   |  12.13    |  301.9    |  0.8019   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  31       | -22.5     |  0.3599   |  1.453    |  10.68    |  141.5    |  0.8378   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  32       | -2.16     |  3.719    |  0.3334   |  10.81    |  486.5    |  0.927    |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warnin

|  57       | -24.3     |  1.173    |  1.385    |  5.299    |  324.5    |  0.8891   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  58       | -1.199    |  4.564    |  0.05     |  11.31    |  326.8    |  0.95     |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  59       | -1.336    |  1.139    |  0.2143   |  13.67    |  325.1    |  0.9314   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  60       | -1.724    |  3.743    |  0.4127   |  12.58    |  321.4    |  0.8192   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  61       | -2.719    |  1.144    |  0.525    |  7.494    |  2

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  86       | -1.2      |  5.0      |  0.05     |  12.28    |  421.1    |  0.95     |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  87       | -4.8      |  0.2084   |  0.7544   |  13.76    |  283.5    |  0.9084   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  88       | -2.246    |  0.1501   |  0.8375   |  11.6     |  65.8     |  0.8786   |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
|  89       | -1.715    |  3.349    |  0.5973   |  14.26    |  85.9     |  0.947    |
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warnin

In [41]:
# Step8. 함수를 최대화하는 하이퍼파라미터 저장
max_params = BO_lgbm.max['params']

# 모델의 조건에 맞게 하이퍼 파라미터 변경
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))

max_params

{'gamma': 2.796753349762953,
 'learning_rate': 0.05,
 'max_depth': 7,
 'n_estimators': 247,
 'subsample': 0.9147760300896326}

In [42]:
# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_clf = LGBMClassifier(**max_params)
lgbm_clf.fit(X_train, y_train)
score = log_loss(y_valid, lgbm_clf.predict_proba(X_valid))
print(score)

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
1.1986424697053901


# Ensemble

In [43]:
# 산술평균, 기하평균, 멱평균 등의 기법을 통해 결과물을 조합한다.
# 성능이 좋은 모델만 선정해서 ensemble
ensemble_pred = np.mean([lgbm_pred, xgb_pred], axis = 0) ; ensemble_pred

array([[3.30011926e-02, 1.62055825e-01, 7.09538703e-01, 2.97657942e-04,
        7.32710698e-03, 8.77795336e-02],
       [1.09682397e-01, 4.06875274e-01, 4.49174735e-01, 3.11441329e-03,
        8.82726964e-03, 2.23259012e-02],
       [5.96610363e-02, 4.49352700e-01, 2.85060285e-01, 3.19434316e-02,
        1.16801308e-01, 5.71812412e-02],
       ...,
       [1.87569732e-01, 3.72636165e-01, 3.78612739e-01, 9.16668941e-03,
        2.31102079e-02, 2.89044885e-02],
       [1.01801767e-01, 1.79867302e-01, 6.91621453e-01, 2.17992162e-03,
        5.75483305e-03, 1.87747142e-02],
       [4.15501320e-02, 5.38112190e-01, 2.48989830e-01, 5.77178364e-03,
        7.99673563e-02, 8.56087018e-02]])

In [44]:
# ensemble한 결과의 score 확인
log_loss(pd.get_dummies(y_valid), ensemble_pred)

1.199437627514092

In [45]:
# 기하평균
from scipy.stats import gmean
ensemble_gmean_pred = gmean([lgbm_pred, xgb_pred]) ; ensemble_gmean_pred

array([[3.28984762e-02, 1.60475934e-01, 7.08386347e-01, 2.10218883e-04,
        7.18437828e-03, 8.67200559e-02],
       [1.02758484e-01, 4.05433789e-01, 4.43077686e-01, 3.05903981e-03,
        8.77403047e-03, 2.21573343e-02],
       [5.88256392e-02, 4.47130423e-01, 2.85040896e-01, 3.13513049e-02,
        1.14916065e-01, 5.70164613e-02],
       ...,
       [1.86919258e-01, 3.72443077e-01, 3.78584666e-01, 9.11259211e-03,
        2.30443519e-02, 2.88516778e-02],
       [1.01702340e-01, 1.74161595e-01, 6.90174179e-01, 2.17577219e-03,
        5.45311259e-03, 1.86332835e-02],
       [4.08847411e-02, 5.37857228e-01, 2.48558590e-01, 5.73177235e-03,
        7.99432431e-02, 8.52214757e-02]])

In [46]:
# ensemble한 결과의 score 확인
log_loss(pd.get_dummies(y_valid), ensemble_gmean_pred)

1.1991715213432996

# Stacking

In [36]:
!pip install vecstack

In [47]:
# stacking
from vecstack import StackingTransformer

lgbm = LGBMClassifier(random_state = 0)
rf = RandomForestClassifier(random_state = 0)
xgb = XGBClassifier(random_state = 0)

models = [
    ('LGBM',lgbm),
    ('RF',rf),
    ("XGB",xgb)
]

stack = StackingTransformer(
    models,                    # stacking할 때 사용할 weak models 설정
    regression=False,         # 회귀문제인지, 분류문제인지
    needs_proba=True,         # 확률로 결과값을 나타낼 것인지
    metric=None,              # 평가지표를 따로 설정할 것인지
    n_folds=3,                 # fold를 몇 번이나 할 것인지
    stratified=True,          # 결과분포를 균등하게 해서 fold할 것인지
    shuffle=True,             # sampling을 무작위로 할 것인지
    random_state=0  
)

stack = stack.fit(X_train, y_train)

In [48]:
# stacking data 생성
S_train = stack.transform(X_train)
S_valid = stack.transform(X_valid)
S_test = stack.transform(data_te)

In [49]:
# pred 6개 * model 3개 = 18
S_train.shape

(184172, 18)

In [50]:
lgbm_meta = LGBMClassifier(random_state = 0)
lgbm_meta.fit(S_train, y_train)

LGBMClassifier(random_state=0)

In [51]:
# stacking model 결과 확인
log_loss(pd.get_dummies(y_valid),lgbm_meta.predict_proba(S_valid))

1.1961952734650805

# Submission

In [29]:
# predict
pred = lgbm_meta.predict_proba(S_test)

In [30]:
submission.loc[:,'F20':'M40'] = pred ; submission

,CLNT_ID,F20,F30,F40,M20,M30,M40
0,263104,0.202407,0.373035,0.352935,0.012003,0.024215,0.035405
1,263105,0.282487,0.468475,0.215442,0.007200,0.012512,0.013884
2,263106,0.105971,0.298823,0.230670,0.039644,0.128882,0.196009
3,263107,0.013075,0.673753,0.249189,0.000173,0.030639,0.033172
4,263108,0.225439,0.427485,0.248473,0.023886,0.041308,0.033409
...,...,...,...,...,...,...,...
112755,375859,0.129696,0.287105,0.164934,0.084612,0.243830,0.089822
112756,375860,0.039808,0.253892,0.662778,0.000879,0.003108,0.039535
112757,375861,0.051852,0.388628,0.485535,0.002221,0.022113,0.049651
112758,375862,0.125973,0.181927,0.243934,0.123927,0.186384,0.137854


In [31]:
t = pd.Timestamp.now()
name = '이지평'
fname = f"{name}_submission_{t.month:11}{t.day:25}{t.hour:15}{t.minute:30}.csv"
submission.to_csv(fname, index = False)